<table style="width:100%">
  <tr>
    <td><center style="font-size:500%;">Representação Distribuida de Palavras (Parte 1)</center></td>
    <td><img src="https://logodownload.org/wp-content/uploads/2015/02/puc-rio-logo.gif" width="100"/></td> 
  </tr>    
</table>



In [1]:
!pip show gensim

Name: gensim
Version: 3.6.0
Summary: Python framework for fast Vector Space Modelling
Home-page: http://radimrehurek.com/gensim
Author: Radim Rehurek
Author-email: me@radimrehurek.com
License: LGPLv2.1
Location: /opt/anaconda3/lib/python3.8/site-packages
Requires: smart-open, scipy, numpy, six
Required-by: 


In [5]:
import numpy as np
import gensim.downloader as api
from sklearn.metrics.pairwise import cosine_similarity

#carrega um modelo pré-treinado (gloVe word vectors)
word_vectors = api.load('glove-wiki-gigaword-50')

# função de word -> vec
word2vec = lambda w: word_vectors.word_vec(w.lower())
#word2vec = lambda w: word_vectors.get_vector(w.lower()) #nova versao fdo gensim 4.1.0

#função de similaridade
similarity = lambda x,y:cosine_similarity(x[np.newaxis,:],y[np.newaxis,:]).squeeze()
#[np.newaxis:1] converte um array 1D para 2D. No caso [50,] para [1,50]. A funcão cosine_similaruty precisa de um
#vetor com n_samples, n_features.

In [6]:
#word_a, word_b = "king", "queen"
word_a, word_b = "blue", "yellow"

vec_a = word2vec(word_a) #transforma em word2vec
vec_b = word2vec(word_b) #transforma em word2vec

#calcula similaridade entre vec_a e vec_b
result = similarity(vec_a, vec_b).squeeze()
print("cosine_similarity( {} , {} ): {:.4f}".format(word_a,word_b,result))

cosine_similarity( blue , yellow ): 0.8321


In [7]:
pos_a, neg_a = "france" , "paris"
pos_a_v = word2vec(pos_a)
neg_a_v = word2vec(neg_a)

pos_b, neg_b = "italy", "rome"
pos_b_v = word2vec(pos_b)
neg_b_v = word2vec(neg_b)

#Quão similar são as palavras:

result = similarity(pos_a_v - neg_a_v, pos_b_v - neg_b_v)
print("cosine_similarity( {} - {} , {} - {} ): {:.4f}".format(pos_a, neg_a, pos_b, neg_b, result))

result = similarity(pos_a_v - neg_a_v, neg_b_v - pos_b_v)
print("cosine_similarity( {} - {} , {} - {} ): {:.4f}".format(pos_a, neg_a, neg_b, pos_b, result))

cosine_similarity( france - paris , italy - rome ): 0.6751
cosine_similarity( france - paris , rome - italy ): -0.6751


In [8]:
#g = word2vec('women')
#g = word2vec('woman') - word2vec('man')
g = word2vec('hispanic') - word2vec('american')

print(g)

[ 0.43528992 -0.89909     0.546175    0.14104998  0.99908     0.13400999
  0.38700008 -0.64735    -0.637116    0.59869    -0.31992    -1.67881
  1.72769    -0.55635     0.077877   -0.58527994 -0.579828   -0.151812
  0.963201   -0.54823995 -0.87078    -0.557199    0.06807998  0.90702
  0.05069999  1.41682     0.40825     0.11616999 -0.799819   -0.278025
 -0.7675998   1.1688      0.08175199 -0.6893699  -0.345674   -0.56614
 -0.76390004 -0.12868002  0.512343    0.13447998 -0.93983996  0.4739176
 -0.76528     0.88283    -1.02021    -0.50976    -0.07191998 -0.13142
  1.35542     0.19582999]


In [9]:
#Nomes masculinos são negativos
print ('Lista de nomes e sua similaridade com o vetor g:')

name_list = ['john', 'marie', 'sophie', 'ronaldo', 'priya', 'rahul', 'danielle', 'reza', 'katy', 'yasmin']

for word in name_list:
    print("{}:{}".format(word, similarity(word2vec(word), g)))

Lista de nomes e sua similaridade com o vetor g:
john:-0.3658096194267273
marie:-0.17237810790538788
sophie:-0.2450234442949295
ronaldo:-0.014981400221586227
priya:0.10761348903179169
rahul:-0.10907028615474701
danielle:-0.13969191908836365
reza:-0.207201287150383
katy:-0.05387929826974869
yasmin:0.07244789600372314


In [10]:
print('Outras palavras e suas similaridades:')
word_list = ['lipstick', 'guns', 'science', 'arts', 'literature', 'warrior','doctor', 'tree', 'receptionist', 
             'technology',  'fashion', 'teacher', 'engineer', 'pilot', 'computer', 'singer']
for word in word_list:
    print("{}:{}".format(word, similarity(word2vec(word), g)))

Outras palavras e suas similaridades:
lipstick:0.028396347537636757
guns:-0.32811033725738525
science:-0.2437894642353058
arts:-0.16481715440750122
literature:-0.15814538300037384
warrior:-0.3153199851512909
doctor:-0.2643163204193115
tree:-0.013971754349768162
receptionist:0.08079687505960464
technology:-0.2412266731262207
fashion:-0.14138080179691315
teacher:-0.1181369498372078
engineer:-0.40797320008277893
pilot:-0.398493230342865
computer:-0.11333662271499634
singer:-0.20428600907325745


In [13]:
def complete_analogy(pos_a, neg_a, pos_b):
    
    pos_a_v, neg_a_v, pos_b_v = word2vec(pos_a), word2vec(neg_a), word2vec(pos_b)
    #calculo a diferença
    d = pos_a_v - neg_a_v
    #vou atualizando o valor no for
    max_cosine_sim = -1            
    best_word = None                   

    for neg_b in word_vectors.vocab:   
    #for neg_b in word_vectors.key_to_index: #nova versao do gensim
        
        if neg_b in [pos_a, neg_a, pos_b] :
            continue
            
        neg_b_v = word2vec(neg_b)
        #busca pos v com neg B
        cosine_sim = similarity(d , pos_b_v - neg_b_v)
        
        if cosine_sim > max_cosine_sim:
            max_cosine_sim = cosine_sim
            best_word = neg_b
            
    return best_word

# as duas palavras fazem a analogia como argentina 

In [14]:
analogies_to_try = [('italy', 'italian', 'argentina'), 
                 ('india', 'delhi', 'japan'), 
                 ('man', 'woman', 'boy'), 
                 ('small', 'smaller', 'large')]

# dadas 3 palavras, encontre a próxima palavra
for analogy in analogies_to_try:
    print ('{} -> {} :: {} -> {}'.format(*analogy, complete_analogy(*analogy))) #* serve para unpacking

italy -> italian :: argentina -> argentine
india -> delhi :: japan -> tokyo
man -> woman :: boy -> girl
small -> smaller :: large -> larger
